<H3> Exercise : Using Orders Data print the following. Use Spark Core APIs (RDD Level). </H3>
<ol>
<li>Total # of orders in each stage (status). </li>
<li>Top 10 customers by # of orders placed. </li>
<li>Count of distinct customer who have placed at least one order. </li>
<li>Customer with the maximum number of closed orders.</li>
</ol>

In [1]:
from pyspark.sql import SparkSession

In [2]:
import getpass
username = getpass.getuser()

In [3]:
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [4]:
spark

In [5]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")

In [17]:
orders_rdd.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [13]:
orders_split = orders_rdd.map(lambda line: line.split(","))

In [18]:
orders_split.take(5)

[['1', '2013-07-25 00:00:00.0', '11599', 'CLOSED'],
 ['2', '2013-07-25 00:00:00.0', '256', 'PENDING_PAYMENT'],
 ['3', '2013-07-25 00:00:00.0', '12111', 'COMPLETE'],
 ['4', '2013-07-25 00:00:00.0', '8827', 'CLOSED'],
 ['5', '2013-07-25 00:00:00.0', '11318', 'COMPLETE']]

In [19]:
order_count_by_status1 = orders_split.map(lambda line: (line[3],1))

In [20]:
order_count_by_status2 = order_count_by_status1.reduceByKey(lambda x,y: x+y)

In [21]:
order_count_by_status2.collect()

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899),
 ('PROCESSING', 8275),
 ('PAYMENT_REVIEW', 729),
 ('PENDING', 7610),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558)]

In [31]:
top_10_customers1 = orders_split.map(lambda line:(line[2],1))

In [32]:
top_10_customers1.take(5)

[('11599', 1), ('256', 1), ('12111', 1), ('8827', 1), ('11318', 1)]

In [33]:
top_10_customers2 = top_10_customers1.reduceByKey(lambda x,y: x+y)

In [34]:
top_10_customers2.take(5)

[('3066', 6), ('3159', 7), ('8135', 11), ('2248', 4), ('6117', 6)]

In [48]:
top_10_customers3 = top_10_customers2.sortBy(lambda x: x[1], ascending=False)

In [50]:
top_10_customers3.take(10)

[('5897', 16),
 ('6316', 16),
 ('12431', 16),
 ('569', 16),
 ('4320', 15),
 ('221', 15),
 ('5624', 15),
 ('5283', 15),
 ('12284', 15),
 ('5654', 15)]

In [55]:
cust_count_atlest_one_order1 = orders_split.map(lambda line:line[2]).distinct()

In [56]:
cust_count_atlest_one_order1.take(5)

['256', '12111', '11318', '7130', '2911']

In [57]:
cust_count_atlest_one_order1.count()

12405

In [58]:
closed_orders = orders_split.filter(lambda line: line[3] == 'CLOSED')

In [59]:
closed_orders.take(5)

[['1', '2013-07-25 00:00:00.0', '11599', 'CLOSED'],
 ['4', '2013-07-25 00:00:00.0', '8827', 'CLOSED'],
 ['12', '2013-07-25 00:00:00.0', '1837', 'CLOSED'],
 ['18', '2013-07-25 00:00:00.0', '1205', 'CLOSED'],
 ['24', '2013-07-25 00:00:00.0', '11441', 'CLOSED']]

In [60]:
cust_count_closed = closed_orders.map(lambda line: (line[2],1))

In [61]:
cust_count_closed_agg = cust_count_closed.reduceByKey(lambda x,y: x+y)

In [64]:
cust_count_closed_agg.sortBy(lambda x: x[1],False).take(1)

[('1833', 6)]